# Working through the "Atacama Soil Microbiome" Tutorial in Qiime2
- The original tutorial is here in the official Qiime2 docs: https://docs.qiime2.org/2021.2/tutorials/atacama-soils/
- This workshop was also adapted from a previous workshop hosted by 
Joslynn Lee: https://cyverse-jupyter-qiime2.readthedocs-hosted.com/en/latest/

### Learning objectives:
- Understand the analysis steps and core concepts of the Qiime2 software
- Generate plots to visualize biodiversity results from metabarcoding data

### How this notebook works:
- Code is fill-in-the-blank, follow along with us as we work through it together!
- Stuck or Lost? We have many points of redemption throughout; just download the output files and visualize them here: https://view.qiime2.org/

### Steps covered:
0. Setting up the Jupyter notebook and input files
1. Demultiplexing paired-end sequences
2. Denoising using DADA2
3. Diversity analyses
4. Bonus: Taxonomic assignment

# 0. Setting up the Jupyter and input files

This first step is only because we are using a Jupyter notebook for this workshop. You will not need to run these few lines of code when working with your own datasets on your computer. 

In [1]:
pip3 install git+https://github.com/regebro/tzlocal
qiime dev refresh-cache

  Cloning https://github.com/regebro/tzlocal to /tmp/pip-req-build-qtg2cfxz
  Running command git clone -q https://github.com/regebro/tzlocal /tmp/pip-req-build-qtg2cfxz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 70 kB 2.0 MB/s eta 0:00:011
  Created wheel for tzlocal: filename=tzlocal-3.0b2.dev0-py3-none-any.whl size=16632 sha256=ebaf36e5cdbd8dd26ef08489404561434f5a244e38d01c0d84d19116af95ba28
  Stored in directory: /tmp/pip-ephem-wheel-cache-0m4s9scp/wheels/ad/0c/96/10852f06599a3c7ad57c560572a82b3365ec56ffab8dbbdcc1
Successfully built tzlocal
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 2.1
    Uninstalling tzlocal-2.1:
      Successfully uninstalled tzlocal-2.1
QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


Generally on your local computer, you will need to activate Qiime2 using conda. Here, we have pre-loaded it, so you will not need to run this command: 

In [2]:
# conda activate qiime2

Now, we will make a directory (folder) for our tutorial, as well as a directory for our raw, paired-end sequences. 
mkdir = make a directory

In [3]:
mkdir qiime2-atacama-tutorial
cd qiime2-atacama-tutorial

Then, we are going to download the sample metadata file from the Qiime2 website. 

In [4]:
wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/sample_metadata.tsv"

--2021-06-09 17:56:14--  https://data.qiime2.org/2021.2/tutorials/atacama-soils/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/1r5nh74kOx7o3YqSjmP0GsWXifaHE-lrPgTFEM7aD87U/export?gid=0&format=tsv [following]
--2021-06-09 17:56:15--  https://docs.google.com/spreadsheets/d/1r5nh74kOx7o3YqSjmP0GsWXifaHE-lrPgTFEM7aD87U/export?gid=0&format=tsv
Resolving docs.google.com (docs.google.com)... 108.177.121.101, 108.177.121.113, 108.177.121.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.121.101|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0k-6o-sheets.googleusercontent.com/export/l5l039s6ni5uumqbsj9o11lmdc/0rr8kskuo8raq6pbbemqok0r7g/1623261375000/103995680502445084602/*/1r5nh74kOx7o3YqSjmP0GsWXifaHE-lrPgTFEM7aD87U?

We are going to put these raw sequences in their own directory. You will download three fastq.gz files, corresponding to the forward, reverse, and barcode (i.e., index) reads. These files contain a subset of the reads in the full data set generated for this study, which allows for the following commands to be run relatively quickly, however, we will perform additional subsampling in this tutorial to further improve the run time.

In [5]:
mkdir emp-paired-end-sequences

wget \
  -O "emp-paired-end-sequences/forward.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/forward.fastq.gz"

wget \
  -O "emp-paired-end-sequences/reverse.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/reverse.fastq.gz"

wget \
  -O "emp-paired-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/barcodes.fastq.gz"

--2021-06-09 17:56:16--  https://data.qiime2.org/2021.2/tutorials/atacama-soils/10p/forward.fastq.gz
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.2/tutorials/atacama-soils/10p/forward.fastq.gz [following]
--2021-06-09 17:56:17--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.2/tutorials/atacama-soils/10p/forward.fastq.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.241.96
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.241.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143193967 (137M) [binary/octet-stream]
Saving to: ‘emp-paired-end-sequences/forward.fastq.gz’

emp-paired-end-sequ 100%[===================>] 136.56M  33.8MB/s    in 4.4s    

2021-06-09 17:56:22 (31.2 MB/s) - ‘emp-paired-end

# 1. Demultiplexing reads

To analyze these data, the sequences that you just downloaded must first be imported into an artifact of type EMPPairedEndSequences.

In [6]:
qiime tools import \
   --type EMPPairedEndSequences \
   --input-path emp-paired-end-sequences \
   --output-path emp-paired-end-sequences.qza

Imported emp-paired-end-sequences as EMPPairedEndDirFmt to emp-paired-end-sequences.qza


We will now demultiplex the sequence reads. This requires the sample metadata file, specifically the column in the file that contains the per-sample barcodes. Here, the file is called barcode-sequence. Because the barcode reads are the reverse complement of those included in the sample metadata file, we can include the --p-rev-comp-mapping-barcodes parameter. One output of this file is looking at a summary of how many sequences were obtained per sample. 

The below code may take a few minutes to run.

In [7]:
qiime demux emp-paired \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --p-rev-comp-mapping-barcodes \
  --i-seqs emp-paired-end-sequences.qza \
  --o-per-sample-sequences demux-full.qza \
  --o-error-correction-details demux-details.qza

Saved SampleData[PairedEndSequencesWithQuality] to: demux-full.qza
Saved ErrorCorrectionDetails to: demux-details.qza


Let's subsample the data - this will speed up the tutorial and run time, and demonstrate the functionality. Other uses of subsampling need to be thought through with reasonable justification. 

In [8]:
qiime demux subsample-paired \
  --i-sequences demux-full.qza \
  --p-fraction 0.3 \
  --o-subsampled-sequences demux-subsample.qza

qiime demux summarize \
  --i-data demux-subsample.qza \
  --o-visualization demux-subsample.qzv

Saved SampleData[PairedEndSequencesWithQuality] to: demux-subsample.qza
Saved Visualization to: demux-subsample.qzv


We can view the summary of our subsampled dataset to examine how many sequence counts were in each sample. Here, there are 75 samples in the data and the last 20 or so of the rows in the table have fewer than 100 reads in them, which can be filtered out of the data:

In [9]:
qiime tools export \
  --input-path demux-subsample.qzv \
  --output-path ./demux-subsample/

qiime demux filter-samples \
  --i-demux demux-subsample.qza \
  --m-metadata-file ./demux-subsample/per-sample-fastq-counts.tsv \
  --p-where 'CAST([forward sequence count] AS INT) > 100' \
  --o-filtered-demux demux.qza

Exported demux-subsample.qzv as Visualization to directory ./demux-subsample/
Saved SampleData[PairedEndSequencesWithQuality] to: demux.qza


# 2. Denoising
We will look at the sequence quality based on 10,000 randomly selected reads from the subsampled and filtered data, then denoise it based on the quality plots. There will be two plots showing quality scores - one for the forward reads and one for the reverse reads. These plots will help us determine what trimming parameters we want to denoise with, using DADA2. 

In this example, we have 150-base forward and reverse reads. We need to keep in mind that the paired reads need to be long enough to overlap so they (the forward and reverse sequences) can merge. We will trim off the first 13 bases of the forward and reverse reads, but will not trim the end of the sequences, to avoid reducing the read length too much. You do not have to keep the trimming or truncation lengths the same for your forward and reverse sequences, but we are doing that here:

The following code may take a few minutes to run. 

In [10]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza


Now, we have artifacts containing the feature table, corresponding feature sequences, and DADA2 denoising statistics. We can generate summaries of these data:

In [11]:
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv
Saved Visualization to: denoising-stats.qzv


Let's view some of the .qzv files on https://view.qiime2.org/ to look at how many sequences remain in each sample.

## 3. Generate a tree for phylogenetic diversity analyses
In addition to counts of features per sample (i.e., the data in the FeatureTable[Frequency] Qiime2 artifact), some biodiversity metrics require a rooted phylogeentic tree relating the features to one another. These phylogenetic diveresity metrics include Faith's Phylogenetic Diversity and weighted and unweighted UniFrac. Phylogenetic trees in Qiime2 are stored as a Phylogeny[Rooted] Qiime2 artifact. We will use the align-to-tree-mafft-fasttree pipeline from the q2-phylogeny plugin. 

First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [12]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


## 4. Alpha and beta diversity analysis
QIIME2 offers diversity analyses through the q2-diversity plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We will first use the core-metrics-phylogenetic method, which rarefies a FeatureTable[Frequency] to a user-specified depth,  computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

Alpha diversity

   - Shannon’s diversity index (a quantitative measure of community richness)
   - Observed Features (a qualitative measure of community richness)
   - Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)
   - Evenness (or Pielou’s Evenness; a measure of community evenness)

Beta diversity

   - Jaccard distance (a qualitative measure of community dissimilarity)
   - Bray-Curtis distance (a quantitative measure of community dissimilarity)
   - unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
   - weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

An important parameter that needs to be provided to this script is --p-sampling-depth, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For example, if you provide --p-sampling-depth 500, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the information presented in the table.qzv file that was created above. Choose a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.

Looking at the table.qzv Qiime2 artifact we just generated, in particular the 'Interative Sample Detail' tab in that visualization, what value would you choose to pass for --p-sampling-depth? How many samples will be excluded from your analysis based on this choice? How many total sequences will you be analyzing in the core-metrics-phylogenetic command?

In [13]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 500 \
  --m-metadata-file sample-metadata.tsv \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

We set the `--p-sampling-depth` parameter to 500. This value was chosen based on the `table.qzv` output: it allowed for most of the samples to be retained while also dropping samples that had a low number of sequences. Losing a disproportionate number of samples from one metadata category is not ideal. You want to strike a balance between total seqeuences analyzed and number of samples retained. 

On an Illumina run, it is common to see a few samples with very low sequence counts. These should typically be removed from analyses by setting a larger value for the sampling depth at this stage.

After computing diversity metrics, we can begin to explore the microbiome community composition of samples in the context of sample metadata. This information is present in the sample metadata file that was downloaded earlier.

We'll first test for association between categorical metadata columns and alpha diversity data. We'll do that here for the Faith Phylogenetic Diversity (a measure of community richness) and evenness metrics. 

In [14]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv
Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


 Next, we will analyze the sample composition in the contet of categorical metadata using PERMANOVA (Anderson, 2001) using the `beta-group-significance` command. The commands will test whether distances between samples within a group are more similar to each other than they are to samples from the other groups. PERMANOVA can also perform pairwise tests that allow you to determine which specific pairs of groups differ from one another. We will do this here with `--p-pairwise`. This command can be slow to run since it's based on permutation test. We will run `beta-group-significance` on specific metadata columns to reduce computation time. Here, will apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows:

In [18]:
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file sample-metadata.tsv \
  --m-metadata-column vegetation \
  --o-visualization core-metrics-results/unweighted-unifrac-vegetation-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/unweighted-unifrac-site-name-significance.qzv


In our dataset, the continuous sample metadata we have for this data set are correlated with vegetation, so we can test for additional associations. We are going to run a further test using the `qiime diversity bioenv` commands. 

In [ ]:
qiime diversity bioenv \
    --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file sample-metadata.tsv \
    --o-visualization core-metrics-results/unweighted-unifrac-bioenv.qzv \

Finally, ordination is a popular approach for exploring microbiome community composition in the context of sample metadata. We can use the Emperor tool to explore principal coordinates (PCoA) plots in the context of sample metadata. While our `core-metrics-phylogenetic` command did already generate some Emperor plots, we want to pass an optional parameter, `--p-custom-axes`, which is very useful for exploring our data. The PCoA results that were used in `core-metrics-phylogeny` are also available, making it easy to generate new visualizations with Emperor. We will generate Emperor plots for unweighted UniFrac and Bray-Curtis so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2, and vegetation start. We will use that last axis to explore how these samples changed across sites based on the environmental variable "depth". 

In [ ]:
qiime feature-table filter-samples \
  --i-table table.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-where "average-soil-temperature" > 0 \
  --o-filtered-table sample-metadata-filtered.tsv


qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes depth \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-depth.qzv

qiime emperor plot \
  --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
  --m-metadata-file sample-metadata.tsv \
  --p-custom-axes depth \
  --o-visualization core-metrics-results/bray-curtis-emperor-depth.qzv